In [1]:
import pandas as pd
from gensim.models import word2vec
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

In [30]:
def get_train_test_csv():
    df_train = pd.read_csv('data/train_data.csv', sep='|', dtype=str)
    df_test = pd.read_csv('data/test_data_A.csv', sep='|', dtype=str)
    df_train = df_train.drop(columns=['label'])
    for name in df_train.columns:
        df_train[name] = df_train[name] + name

    df_test = df_test.drop(columns=['id'])
    for name in df_test.columns:
        df_test[name] = df_test[name] + name
    data = pd.concat([df_train, df_test], ignore_index=True)
    data.to_csv('data/train_test.csv', sep=' ', index=0, header=0)
# get_train_test_csv()

In [58]:
def get_embedding():
    path = f'word2vec/word2vec_kv.kv'
    wv = KeyedVectors.load(path, mmap='r')
    feature_tokens = list(wv.vocab.keys())
#     tokenizer = Tokenizer(num_words=1187210)
    num_keys = len(list(wv.vocab.keys()))
    print('number of keys', num_keys)
    tokenizer = Tokenizer(num_words=num_keys,filters='\n')#去除下划线
    tokenizer.fit_on_texts(feature_tokens)
    embedding_dim = 512
    embedding_matrix = np.random.randn(
        len(feature_tokens)+1, embedding_dim)
    for word, i in tokenizer.word_index.items():
        try:
            embedding_vector = wv[word]
        except:
            print(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            
    return embedding_matrix
# embedding_matrix = get_embedding()
# np.save('word2vec/embedding_matrix.npy', embedding_matrix)

In [71]:
def get_train_test_npy():
    path = f'word2vec/word2vec_kv.kv'
    wv = KeyedVectors.load(path, mmap='r')
    feature_tokens = list(wv.vocab.keys())
    num_keys = len(list(wv.vocab.keys()))
    print('number of keys', num_keys)
    tokenizer = Tokenizer(num_words=num_keys,filters='\n')#去除_ ^
    tokenizer.fit_on_texts(feature_tokens)
    
    feature_seq = []

    with open('data/train_test.csv') as f:
        for text in f:
            feature_seq.append(text.strip())

    sequences = tokenizer.texts_to_sequences(feature_seq[:41907133])
    x_train = pad_sequences(
        sequences, maxlen=36, padding='post')
    print(x_train.shape)
    np.save('data/x_train.npy', x_train)

    sequences = tokenizer.texts_to_sequences(feature_seq[41907133:])
    x_test = pad_sequences(
        sequences, maxlen=36, padding='post') 
    print(x_test.shape)
    np.save('data/x_test.npy', x_test)
# get_train_test_npy()

In [74]:
def get_label_npy():
    train = pd.read_csv('data/train_data.csv', sep='|', dtype=str)
    label = train['label']
    label = to_categorical(label)
    np.save('data/label.npy',label)
# get_label_npy()

In [2]:
label = np.load('data/label.npy', allow_pickle=True)

In [3]:
label.shape

(41907133, 2)

In [4]:
label[:10]

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)